In [1]:
from PIL import Image
import numpy as np
from collections import Counter
import colorsys

# Define rainbow colors in RGB
RAINBOW_COLORS = {
    'Red': (255, 0, 0),
    'Orange': (255, 165, 0),
    'Yellow': (255, 255, 0),
    'Green': (0, 255, 0),
    'Blue': (0, 0, 255),
    'Indigo': (75, 0, 130),
    'Violet': (238, 130, 238)
}

def calculate_average_brightness(image):
    grayscale_image = image.convert('L')  # Convert to grayscale
    pixels = np.array(grayscale_image)
    return np.mean(pixels)  # Return average brightness

def calculate_saturation(image):
    pixels = np.array(image)
    hsv_pixels = [colorsys.rgb_to_hsv(r/255.0, g/255.0, b/255.0) for r, g, b in pixels.reshape(-1, 3)]
    saturations = [s for h, s, v in hsv_pixels]
    return np.mean(saturations)  # Return average saturation

def find_rainbow_colors(image, rainbow_colors):
    # Convert image to RGB
    pixels = np.array(image.convert('RGB'))
    
    # Flatten the image array and create a color Counter
    flattened_pixels = pixels.reshape(-1, 3)
    pixel_counter = Counter(map(tuple, flattened_pixels))

    # Compare each pixel with rainbow colors
    rainbow_frequency = {color: 0 for color in rainbow_colors.keys()}
    for pixel, count in pixel_counter.items():
        for color_name, color_value in rainbow_colors.items():
            if np.linalg.norm(np.array(pixel) - np.array(color_value)) < 50:  # Tolerance for color matching
                rainbow_frequency[color_name] += count
    
    # Order colors by frequency
    ordered_colors = sorted(rainbow_frequency.items(), key=lambda x: x[1], reverse=True)
    return ordered_colors

def assign_music_features(brightness, ordered_colors, saturation):
    # Assign pitch based on brightness (A4 = 440Hz is middle range)
    base_pitch = 440  # Middle A pitch
    pitch = int(base_pitch * (brightness / 255))  # Scale brightness to pitch range
    
    # Assign sequence of tones based on color order (map each color to a musical note)
    color_to_note = {
        'Red': 'C',
        'Orange': 'D',
        'Yellow': 'E',
        'Green': 'F',
        'Blue': 'G',
        'Indigo': 'A',
        'Violet': 'B'
    }
    tone_sequence = [color_to_note[color] for color, freq in ordered_colors if freq > 0]

    # Assign tempo based on saturation (higher saturation, faster tempo)
    base_tempo = 120  # Base tempo in BPM
    tempo = int(base_tempo * saturation)  # Scale saturation to tempo
    
    return pitch, tone_sequence, tempo

def process_image(image_path):
    image = Image.open(image_path)
    
    # Extract features
    brightness = calculate_average_brightness(image)
    saturation = calculate_saturation(image)
    ordered_colors = find_rainbow_colors(image, RAINBOW_COLORS)
    
    # Assign music features
    pitch, tone_sequence, tempo = assign_music_features(brightness, ordered_colors, saturation)
    
    # Print the results
    print(f"Average Brightness: {brightness}")
    print(f"Color Frequencies: {ordered_colors}")
    print(f"Average Saturation: {saturation}")
    print(f"Assigned Pitch (Hz): {pitch}")
    print(f"Tone Sequence: {tone_sequence}")
    print(f"Assigned Tempo (BPM): {tempo}")
    
    return {
        "brightness": brightness,
        "ordered_colors": ordered_colors,
        "saturation": saturation,
        "pitch": pitch,
        "tone_sequence": tone_sequence,
        "tempo": tempo
    }

# Example usage
image_path = '001_pillars_of_creation_new.jpg'  # Replace with your image path
process_image(image_path)


Average Brightness: 81.31575482262704
Color Frequencies: [('Violet', 851), ('Orange', 56), ('Indigo', 33), ('Red', 0), ('Yellow', 0), ('Green', 0), ('Blue', 0)]
Average Saturation: 0.394141212077037
Assigned Pitch (Hz): 140
Tone Sequence: ['B', 'D', 'A']
Assigned Tempo (BPM): 47


{'brightness': 81.31575482262704,
 'ordered_colors': [('Violet', 851),
  ('Orange', 56),
  ('Indigo', 33),
  ('Red', 0),
  ('Yellow', 0),
  ('Green', 0),
  ('Blue', 0)],
 'saturation': 0.394141212077037,
 'pitch': 140,
 'tone_sequence': ['B', 'D', 'A'],
 'tempo': 47}